In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

import imp
try:
    imp.find_module('jupyterplot')
    from jupyterplot import ProgressPlot
except ImportError:
    !pip install jupyterplot
    from jupyterplot import ProgressPlot

import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets as D
from torchvision import transforms as T

from utils import train_step, test_step
from utils import get_cifar10_dataset, make_dataloader
from utils import simulate_scheduler

# 지금까지 배운 내용을 조합해 자신의 모델 훈련해보기

### Model
- Convolution layer 개수: input / output feature의 크기가 같도록 하면 늘릴 수 있음
- Convolution layer의 out_channels 조절해보기
- Fully connected layer(nn.Linear)의 features 조절해보기 / 개수 조절해보기

In [ ]:
class MyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        ##### layers here ####
    
    def forward(self, x):
        #### convolutions ####
        
        ######################
        x = x.flatten(1)
        ####      FCs     ####
        
        ######################
        return x

print(MyCNN())

### Hyperparameters

- Epoch 수
- learning rate
- (optional) batch_size

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
momentum = 0.9
phases = ['train', 'test']

num_epochs = 1
learning_rate = 0.01
batch_size = 64

### Data Augmentation

In [ ]:
data_augmentation = False

dataset = get_cifar10_dataset(random_crop=data_augmentation)
loader = make_dataloader(dataset, batch_size)

# Learning rate scheduling
- gamma: 한 epoch당 얼만큼 줄일 것인가. 숫자가 낮을수록 빠르게 줄어듦

In [ ]:
gamma = 0.9
lrs = simulate_scheduler(gamma, num_epochs)
plt.plot(lrs)
plt.xlabel('epoch')
plt.ylabel('learning rate')
plt.show()

In [ ]:
model = MyCNN().to(device)
optimizer = torch.optim.SGD(model.parameters(), learning_rate, 0.9)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma)

pp = ProgressPlot(
    plot_names=phases,
    line_names=['loss', 'accuracy'],
    x_lim=[0, None],
    x_label='Iteration',
    y_lim=[[0, None], [0, 100]]
)

accuracy = 0
for epoch in range(num_epochs):
    for inputs, target in loader['train']:
        loss = train_step(model, inputs, target, optimizer, criterion, device)
        pp.update([[loss, -1], [-500, accuracy]])
    
    corrects = 0
    for inputs, target in loader['test']:
        output, _ = test_step(model, inputs, target, device=device)
        corrects += (output.argmax(1).cpu() == target).sum().item()
    accuracy = corrects / len(dataset['test']) * 100
    
    print(f'Epoch: {epoch+1} accuracy {accuracy:.2f}')